In [1]:
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    cluster=LocalCluster(host="tcp://127.0.0.1:2456",dashboard_address="127.0.0.1:2467",n_workers=4)
    client = Client(cluster)

In [155]:
import numpy as np
import pandas as pd
import xarray as xr
import math
import dask

import skimage.feature
import skimage.segmentation
import scipy.ndimage as ndi

import glob

import multiprocessing
from multiprocessing import Pool, Manager

In [156]:
def pre_process(U,V):
    try:
        U.longitude
        U.latitude
    except:
        V = V.rename({'lon':'longitude'})
        V = V.rename({'lat':'latitude'})
        U = U.rename({'lon':'longitude'})
        U = U.rename({'lat':'latitude'})
        U.longitude
        U.latitude
    try:
        test = U.uas
    except:
        try:
            test = U.ua
            U = U.rename({'ua':'uas'})
            V = V.rename({'va':'vas'})
        except:
            U = U.rename({'uwnd':'uas'})
            V = V.rename({'vwnd':'vas'})
    try:
        test = U.level
    except:
        try:
            test = U.plev
            U = U.rename({'plev':'level'})
            V = V.rename({'plev':'level'})
        except:
            U = U.rename({'height':'level'})
            V = V.rename({'height':'level'})
    return U,V

In [157]:
def front_watershed(U,V,lon,lat):
    print('front_watershed')
    front = U.copy()
    front['uas'] = front.uas*0
    front = front.rename({'uas':'x'})
    front['U2'] = U.uas #now
    front['V2'] = V.vas #now
    front['U1'] = xr.DataArray(np.concatenate([U.uas.values[:2]*np.nan,U.uas.values[:-2]]),dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat})
    front['V1'] = xr.DataArray(np.concatenate([V.vas.values[:2]*np.nan,V.vas.values[:-2]]),dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat})
    front = front.sel(time=front.time[2:])
    x = xr.where(front['U1'].values>0,front['U1'].values/front['U1'].values,np.nan)
    x = xr.where(front['V1'].values<0,x,np.nan)
    x = xr.where(front['U2'].values>0,x,np.nan)
    x = xr.where(front['V2'].values>0,x,np.nan)
    x = xr.where(front['V2'].values - front['V1'].values >  2.0 ,x,np.nan)
    front['x'] = xr.DataArray(x,dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat}) #were above frontal criteria are met
    front['front'] = xr.DataArray(x*np.nan,dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat}) #empty for output
    x = np.nan_to_num(x).astype('int64')
    for d in range(2,len(front.time)): #watershed segmentation to label each front
        front_idx = skimage.feature.peak_local_max(x[d], footprint=np.ones((3, 3)),labels=x[d])
        front_mask = np.full_like(x[d], fill_value = False)
        front_mask[tuple(front_idx.T)] = True
        markers = ndi.label(front_mask)[0]
        obj = skimage.segmentation.watershed(-x[d], markers, mask=front_mask)
        z = xr.DataArray(obj, dims=("lat", "lon"), coords={"lon": lon , "lat":lat})
        front['front'][d] = z
    front['front'] = front.front/front.front
    return front

In [158]:
def getfront(frontin,dx='dx',dy='dy',lat='lat',lon='lon'):
    print('getfront')
    front = frontin
    front = front.sel(time=front.time[2:])
    for d in range(len(front.time)):
        winners=[]
        if math.isnan(np.nanmax(front.front[d]))==False:
            for z in range(1,int(np.nanmax(front.front[d]))+1):
                ids =  np.argwhere(front.front[d].values==z)
                latsid = [item[0] for item in ids]
                lonsid = [item[1] for item in ids]
                lats = [lat[i] for i in latsid]
                lons = [lon[i] for i in lonsid]
                mlat = np.mean(lats)
                imlat = np.argmin((lat-mlat)**2) #mean lat
                #area = dy*np.sum(dx[latsi[z]]) #area not sure why though not used again?
                xlen = dx[imlat]*4*np.std(lons)
                ylen = dy*4*np.std(lats)
                length = np.sqrt((ylen**2)+(xlen**2))  #length
                if length>500.0:   #must be greater than 500km
                    winners.append(z)
            front['front'][d] = front.front[d].where(np.isin(front.front[d],winners))
    return front


In [159]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/rcp85/*")
    avail={}
    for model in models:
        uas = glob.glob(str(model)+"/r1i1p1/3hr/native/uas*")
        vas = glob.glob(str(model)+"/r1i1p1/3hr/native/vas*")
        try:
            test = uas[0]
            avail[model.split('/')[-1]] = {'uas':uas,'vas':vas}
        except:
             pass
    return avail

In [160]:
def compute(dict_in ,key):
    print(key)
    U = xr.open_mfdataset(dict_in['uas'])
    V = xr.open_mfdataset(dict_in['vas'])
    U,V = pre_process(U,V)
    U = U.sel(time=slice('2030-01-01', '2045-12-01')) #1950 to last available
    V = V.sel(time=slice('2030-01-01', '2045-12-01'))
    U = U.sel(latitude = slice(-75,-15))
    U = xr.concat([U.sel(longitude = slice(0,30)),U.sel(longitude = slice(320,360))],dim='longitude')
    V = V.sel(latitude = slice(-75,-15))
    V = xr.concat([V.sel(longitude = slice(0,30)),V.sel(longitude = slice(320,360))],dim='longitude')
    V.coords['longitude'] = (V.coords['longitude'] + 180) % 360 - 180
    U.coords['longitude'] = (U.coords['longitude'] + 180) % 360 - 180
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        U = U.sortby(U.longitude).load()
        V = V.sortby(U.longitude).load()
    lon = U.longitude.values
    nlon = len(lon)
    lat = U.latitude.values
    nlat = len(lat)
    front = front_watershed(U,V,lon,lat)
    U.close()
    V.close()
    dx = np.cos(lat*math.pi/180.0)*2*math.pi*6370/360*(lon[-1]-lon[1])/(nlon-1)
    dy = ((lat[2]-lat[1])/180.0)*6370*math.pi
    front = getfront(front,dx,dy,lat,lon)
    front = front.fillna(0)
    front = front.front.resample(time = 'QS-DEC').sum()
    front.to_netcdf(path='Files/'+str(key)+'_seasonal_fronts.nc', mode='w')
    return front

In [161]:
avail = get_files()

In [162]:
for model in avail:
    try:
        compute(avail[model], model)
    except:
        print(model, '- failed')

GFDL-CM3
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


GFDL-ESM2G
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


IPSL-CM5A-LR
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


GFDL-ESM2M
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


IPSL-CM5A-MR
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


In [163]:
files = ['CMIP5/output1/MIROC/MIROC-ESM/rcp85/3hr/atmos/3hr/r1i1p1/', 
'CMIP5/output1/MIROC/MIROC-ESM-CHEM/rcp85/3hr/atmos/3hr/r1i1p1/',
'CMIP5/output1/MIROC/MIROC5/rcp85/3hr/atmos/3hr/r1i1p1/',
'CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/3hr/atmos/3hr/r1i1p1/ ',
'CMIP5/output1/MRI/MRI-CGCM3/rcp85/3hr/atmos/3hr/r1i1p1/',
'CMIP5/output1/MRI/MRI-ESM1/rcp85/3hr/atmos/3hr/r1i1p1/',
'CMIP5/output1/LASG-IAP/FGOALS-s2/rcp85/3hr/atmos/3hr/r1i1p1/', 
'CMIP5/output1/BNU/BNU-ESM/rcp85/3hr/atmos/3hr/r1i1p1/',
'CMIP5/output1/CMCC/CMCC-CM/rcp85/3hr/atmos/3hr/r1i1p1/']
path  = '/terra/data/incoming/cmip5/'
others = {}
for file in files:
    model = file.split('/')[3]
    uas = glob.glob(str(path)+str(file)+"uas/*")
    vas = glob.glob(str(path)+str(file)+"vas/*")
    others[model] = {'uas':uas,'vas':vas}

In [164]:
others['CNRM-CM5']['uas'] = glob.glob('/terra/data/incoming/cmip5/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/3hr/atmos/3hr/r1i1p1/uas/*')
others['CNRM-CM5']['vas'] = glob.glob('/terra/data/incoming/cmip5/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/3hr/atmos/3hr/r1i1p1/vas/*')

In [167]:
for model in others:
    try:
        compute(others[model], model)
    except:
        print(model, '- failed')

MIROC-ESM
front_watershed


<ipython-input-157-08aa31cbf7e7>:11: RuntimeWarning: invalid value encountered in true_divide
  x = xr.where(front['U1'].values>0,front['U1'].values/front['U1'].values,np.nan)


getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


MIROC-ESM-CHEM
front_watershed


...

BNU-ESM
front_watershed
getfront


<ipython-input-158-c98648332448>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


CMCC-CM
front_watershed


<ipython-input-157-08aa31cbf7e7>:11: RuntimeWarning: invalid value encountered in true_divide
  x = xr.where(front['U1'].values>0,front['U1'].values/front['U1'].values,np.nan)


getfront


In [171]:
xr.open_mfdataset(others['CNRM-CM5']['uas'])

/share/apps/anaconda3/envs/dask/lib/python3.8/site-packages/xarray/coding/times.py:517: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/share/apps/anaconda3/envs/dask/lib/python3.8/site-packages/numpy/core/_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 128, lon: 256, time: 233736)
Coordinates:
  * time       (time) object 2026-01-01T03:00:00 ... 2301-01-01 00:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 10.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(14608, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(14608, 128, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(14608, 256, 2), meta=np.ndarray>
    uas        (time, lat, lon) float32 dask.array<chunksize=(14608, 128, 256), meta=np.ndarray>
Attributes: (12/28)
    institution:            CNRM (Centre National de Recherches Meteorologiqu...
    institute_id:           CNRM-CERFACS
    experiment_id:          rcp85
    source:                 CNRM-CM5 2010 Atmosphere: ARPEGE-Climat (V5.2.1, ...
    model_id:               CNRM-CM5
    forcing:                GHG, SA, Sl, BC, OC (Sl variation as a 13-year cy...
    ...                     ...
    table_id:               Table 3hr (31 January 2011) af70bedab182cc487deaa...
    title:                  CNRM-CM5 model output prepared for CMIP5 RCP8.5
    parent_experiment:      pre-industrial control
    modeling_realm:         atmos
    realization:            1
    cmor_version:           2.5.3

In [166]:
client.close()